<a href="https://colab.research.google.com/github/HJJunn/DeepLearning---NLP/blob/main/18_3_KoBERT%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_KorNLI_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KoBERT를 이용한 KorNLI 풀어보기 (다중 클래스 분류)

## 1) 데이터 로드 및 정제

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
!pip install transformers

In [3]:
 # 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")
 # 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")
 # 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x7e084d899bd0>)

In [4]:
 train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
 train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
 val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
 test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [5]:
#결합 후 섞기
train_data = pd.concat([train_snli, train_xnli], ignore_index=True)
train_data = train_data.sample(frac = 1)

In [6]:
train_data.head()

,sentence1,sentence2,gold_label
188198,다과대 앞에 서 있는 남녀,노인이 약을 먹고 있다.,contradiction
469632,한 여성이 다른 사람과 함께 보도 위를 걷고 있다.,여자가 밖으로 나가고 있다.,entailment
210729,검은 옷을 입은 남자들이 흰 옷을 입은 남자를 쫓아다닌다.,카페의 여자들,contradiction
783515,그러나 우리는 인구 밀도가 고정 비용 차이에 대한 유일한 설명이 아니라는 것을 보아왔다.,인구밀도만이 고정비용 차이에 대한 유일한 설명인 것 같다.,contradiction
772742,그는 총 벨트를 매고 트윈 홀스터를 편안하게 내려놓았다.,그의 총 벨트가 가능한 한 가장 빠른 추첨을 할 수 있는 위치에 앉아 있었다.,neutral


In [7]:
val_data.head()

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [8]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


In [9]:
# 중복, 결측값 제거
def drop_na_and_duplicates(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop = True)
    return df

In [10]:
train_data = drop_na_and_duplicates(train_data)
val_data = drop_na_and_duplicates(val_data)
test_data = drop_na_and_duplicates(test_data)

In [11]:
print("훈련용 샘플 개수:", len(train_data))
print("검증용 샘플 개수:", len(val_data))
print("테스트용 샘플 개수:", len(test_data))

훈련용 샘플 개수: 941814
검증용 샘플 개수: 2490
테스트용 샘플 개수: 5010


## 2) BERT의 입력

In [12]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [13]:
max_seq_len = 128
sent1 = train_data['sentence1'].iloc[0]
sent2 = train_data['sentence2'].iloc[0]

print('문장:', sent1)
print('문장:', sent2)

문장: 다과대 앞에 서 있는 남녀
문장: 노인이 약을 먹고 있다.


* encoder_plus : input_ids, token_type_ids, attention_mask를 자동으로 생성해서 리턴

In [14]:
encoding_result = tokenizer.encode_plus(sent1, sent2, max_length = max_seq_len, padding = 'max_length', truncation = True)
print(encoding_result['input_ids'])

[2, 809, 2145, 2104, 1388, 2170, 1258, 1513, 2259, 5969, 3, 4662, 2052, 1397, 2069, 1059, 2088, 1513, 2062, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


* 정수 2: [CLS]
* 정수 3: [SEP]
* 정수 0: [PAD]

In [15]:
#세그먼트 임베딩
print(encoding_result['token_type_ids'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
print(encoding_result['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
#모든 데이터 전처리

def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
    input_ids, attention_masks, token_type_ids = [],[],[]

    for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total = len(sent_list1)):
        encoding_result = tokenizer.encode_plus(sent1, sent2, max_length = max_seq_len, padding = 'max_length', truncation = True)

        input_ids.append(encoding_result['input_ids'])
        attention_masks.append(encoding_result['attention_mask'])
        token_type_ids.append(encoding_result['token_type_ids'])

    input_ids = np.array(input_ids, dtype = int)
    attention_masks = np.array(attention_masks, dtype = int)
    token_type_ids = np.array(token_type_ids, dtype = int)

    return (input_ids, attention_masks, token_type_ids)

In [18]:
X_train = convert_examples_to_features(
    train_data['sentence1'], train_data['sentence2'],
    max_seq_len = max_seq_len, tokenizer = tokenizer)

  0%|          | 2639/941814 [00:01<06:25, 2437.89it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  0%|          | 2884/941814 [00:01<06:41, 2336.81it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have 

In [19]:
input_id = X_train[0][0]
attention_mask = X_train[1][0]
token_type_id = X_train[2][0]

print('단어에대한정수인코딩:',input_id)
print('어텐션마스크:',attention_mask)
print('세그먼트인코딩:',token_type_id)
print('각인코딩의길이:', len(input_id))
print('정수인코딩복원:',tokenizer.decode(input_id))

단어에대한정수인코딩: [   2  809 2145 2104 1388 2170 1258 1513 2259 5969    3 4662 2052 1397
 2069 1059 2088 1513 2062   18    3    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
어텐션마스크: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
세그먼트인코딩: [0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 

In [20]:
X_val = convert_examples_to_features(
    val_data['sentence1'], val_data['sentence2'],
    max_seq_len = max_seq_len, tokenizer = tokenizer
)

100%|██████████| 2490/2490 [00:01<00:00, 2337.67it/s]


In [21]:
X_test = convert_examples_to_features(
    test_data['sentence1'], test_data['sentence2'],
    max_seq_len = max_seq_len, tokenizer = tokenizer
)

100%|██████████| 5010/5010 [00:02<00:00, 2327.40it/s]


In [28]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

In [29]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_val = idx_encode.transform(val_label) # 고유한 정수로 변환
y_test = idx_encode.transform(test_label) # 고유한 정수로 변환

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{np.str_('contradiction'): np.int64(0), np.str_('entailment'): np.int64(1), np.str_('neutral'): np.int64(2)}
{np.int64(0): np.str_('contradiction'), np.int64(1): np.str_('entailment'), np.int64(2): np.str_('neutral')}


In [23]:
print("변환전:", train_label[:5])
print("변환후", y_train[:5])

변환전: ['contradiction', 'entailment', 'contradiction', 'contradiction', 'neutral']
변환후 [0 1 0 0 2]


## 3) BERT를 이용한 다중 클래스 분류 모델

* [CLS] 토큰의 출력층에 소프트맥스 함수 출력층 연결
* outputs[1]을 통해 CLS 토큰 위치 벡터 사용

In [30]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [31]:
model = TFBertForSequenceClassification("klue/bert-base", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.embeddings.position_ids', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [34]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=2)

model.fit(
    X_train, y_train, epochs=1, batch_size=128, validation_data = (X_val, y_val),
    callbacks = [early_stopping]
)

7358/7358 ━━━━━━━━━━━━━━━━━━━━ 6884s 935ms/step - accuracy: 0.4452 - loss: 1.0531 - val_accuracy: 0.4494 - val_loss: 1.0430


In [35]:
print("테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size = 1024)[1]))

5/5 ━━━━━━━━━━━━━━━━━━━━ 71s 11s/step - accuracy: 0.4693 - loss: 1.0346
테스트 정확도: 0.4613
